In [1]:
import ants
#import numpy as np
import os

In [2]:
_atlas_fiber_DTI_path = "tissue_reconstruction/data/FSL_HCP1065_tensor_1mm_space-HPC-AntsIndexSpace_SRI.nii.gz"
_atlas_t1_path = "tissue_reconstruction/data/sub-mni152_t1-inside-brain_space-sri.nii.gz"
_patient_path = "tissue_reconstruction/data/sample_patient.nii.gz"


tensor_image = ants.image_read(_atlas_fiber_DTI_path)
atlas_t1 = ants.image_read(_atlas_t1_path)
patient = ants.image_read(_patient_path)

reg = ants.registration(patient, atlas_t1, "SyN")

transform = reg['fwdtransforms']



In [3]:
print(tensor_image)


ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 6
	 Dimensions : (240, 240, 155)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (239.0, 0.0, 0.0)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]



In [5]:
def transform_tensor_img(fixed_image, dti_img, fwd_transform_paths):
    # 1. Split into Six components
    dti_components = ants.split_channels(dti_img)
    #print(dti_components)

    # 2. Transform every component
    transformed_components = []
    for component in dti_components:
        transformed_component = ants.apply_transforms(fixed=fixed_image, 
                                                    moving=component, 
                                                    transformlist=fwd_transform_paths)
        transformed_components.append(transformed_component)
    transformed_dti_img = ants.merge_channels(transformed_components)

    print(transformed_dti_img)
    
    # 3. Reorient Image ?!
    transformed_dti_save_path = "./transformed_dti.nii.gz"
    transformed_rotated_dti_save_path = "./transformed_rotated_dti.nii.gz"

    ants.image_write(transformed_dti_img, transformed_dti_save_path)

    reorientTensor = f"ReorientTensorImage 3 {transformed_dti_save_path} {transformed_rotated_dti_save_path} {fwd_transform_paths[0]}"
    os.system(reorientTensor) 
    print(reorientTensor)



transform_tensor_img(patient, tensor_image, transform)

ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 6
	 Dimensions : (240, 240, 155)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (-0.0, -239.0, 0.0)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

ReorientTensorImage 3 ./transformed_dti.nii.gz ./transformed_rotated_dti.nii.gz /tmp/tmpjhjdodjs1Warp.nii.gz


sh: 1: ReorientTensorImage: not found
